# Anomaly Detection

## Note:
It is recommended to run the code using a GPU. To do this, go to Runtime > Change runtime type > Hardware Accelerator > select GPU.

It may be the case that a GPU is not available, in which case, use a default CPU ("None" hardware accelerator). The code will still work without a GPU, but may run much slower.

### Contributers:
Jason Ding

## Pre-requisites

In the following cells, we are installing and importing the necessary libaries and downloading the classification model.

In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # It says this is to identify the specific API call causing the error


!wget https://github.com/hendrycks/outlier-exposure/raw/master/CIFAR/snapshots/baseline/cifar10_wrn_baseline_epoch_99.pt
!wget https://raw.githubusercontent.com/hendrycks/pre-training/master/robustness/adversarial/models/wrn_with_pen.py
# !pip3 install torchvision==0.12.0 # Because this version of Pytorch does not work well w/ current version of CUDA 

# Install the latest version of pytorch (without specifying the CUDA version )
!pip install torch torchvision

# !pip install torch torchvision -f https://download.pytorch.org/whl/cu102/torch_stable.html


--2023-05-08 00:49:00--  https://github.com/hendrycks/outlier-exposure/raw/master/CIFAR/snapshots/baseline/cifar10_wrn_baseline_epoch_99.pt
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hendrycks/outlier-exposure/master/CIFAR/snapshots/baseline/cifar10_wrn_baseline_epoch_99.pt [following]
--2023-05-08 00:49:00--  https://raw.githubusercontent.com/hendrycks/outlier-exposure/master/CIFAR/snapshots/baseline/cifar10_wrn_baseline_epoch_99.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9037421 (8.6M) [application/octet-stream]
Saving to: ‘cifar10_wrn_baseline_epoch_99.pt’

cifar10_wrn_baselin 10

In [2]:
# Uninstall the current version of Pytorch 
# !pip uninstall torch torchvision -y


In [3]:
# Install the latest version of pytorch (without specifying the CUDA version )
# !pip install torch torchvision

# !pip install torch torchvision -f https://download.pytorch.org/whl/cu102/torch_stable.html


In [4]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())
import math
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torchvision import datasets
import numpy as np
import torch.nn.functional as F
import sklearn.metrics as sk   # TO calculate AUROC score w/ sklearn 
from wrn_with_pen import WideResNet
prefetch = 2
print("PyTorch Version: ", torch.__version__)
print("CUDA Version: ", torch.version.cuda)  # We want to know whether the GPU we are using is compatible to the current Pytorch version 

PyTorch version: 2.0.0+cu118
CUDA version: 11.8
cuDNN version: 8700
PyTorch Version:  2.0.0+cu118
CUDA Version:  11.8


In [5]:
# check the GPU on your Colab instance
!nvidia-smi  
# and here to check CUDA compatibility for specific GPU model 

Mon May  8 00:49:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Model and Data Loading

In the following cells, we are loading the CIFAR-10 model. Additionally, we are retrieving CIFAR-10 and out-of-distribution datasets.

In [6]:
# Load CIFAR-10 model

net = WideResNet(depth=40, num_classes=10, widen_factor=2, dropRate=0.3)

if torch.cuda.is_available():
    net.load_state_dict(torch.load('cifar10_wrn_baseline_epoch_99.pt'))
    net.eval()
    net.cuda()
else:
    net.load_state_dict(torch.load('cifar10_wrn_baseline_epoch_99.pt', map_location=torch.device('cpu')))
    net.eval()

In [7]:
# /////////////// Loading Datasets ///////////////
mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

# /////////////// CIFAR-10 ///////////////

data_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])

cifar_10_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=data_transform
)

cifar10_data = cifar_10_data
cifar10_loader = torch.utils.data.DataLoader(cifar10_data, batch_size=200, shuffle=False,
                                          num_workers=prefetch, pin_memory=True)
ood_num_examples = len(cifar10_data) // 5

# /////////////// Rademacher Noise ///////////////

dummy_targets = torch.ones(ood_num_examples)
ood_data = torch.from_numpy(np.random.binomial(
    n=1, p=0.5, size=(ood_num_examples, 3, 32, 32)).astype(np.float32)) * 2 - 1
rademacher_ood_data = torch.utils.data.TensorDataset(ood_data, dummy_targets)
rademacher_ood_loader = torch.utils.data.DataLoader(rademacher_ood_data, batch_size=200, shuffle=True)

# /////////////// SVHN ///////////////

data_transform = transforms.Compose([transforms.Resize(32), transforms.ToTensor(), transforms.Normalize(mean, std)])

svhn_ood_data = torchvision.datasets.SVHN(root = "data", 
                          split="test",
                          transform = data_transform, 
                          download = True)

svhn_ood_loader = torch.utils.data.DataLoader(svhn_ood_data, batch_size=200, shuffle=True,
                                         num_workers=prefetch, pin_memory=True)

# /////////////// DTD ///////////////

# data_transform = transforms.Compose([transforms.Resize(32), transforms.CenterCrop(32),
#                                      transforms.ToTensor(), transforms.Normalize(mean, std)])

# dtd_ood_data = torchvision.datasets.DTD(root = "data", 
#                           split="test",
#                           transform = data_transform, 
#                           download = True)

# dtd_ood_loader = torch.utils.data.DataLoader(dtd_ood_data, batch_size=200, shuffle=True,
#                                          num_workers=prefetch, pin_memory=True)

# /////////////// CIFAR-100 ///////////////

data_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])

cifar100_ood_data = datasets.CIFAR100(
    root="data",
    train=False,
    download=True,
    transform=data_transform
)

cifar100_ood_loader = torch.utils.data.DataLoader(cifar100_ood_data, batch_size=200, shuffle=True,
                                         num_workers=prefetch, pin_memory=True)

100%|██████████| 170498071/170498071 [00:05<00:00, 29582287.59it/s]


Extracting data/cifar-10-python.tar.gz to data


100%|██████████| 64275384/64275384 [00:06<00:00, 9458314.87it/s] 


100%|██████████| 169001437/169001437 [00:05<00:00, 29636373.48it/s]


Extracting data/cifar-100-python.tar.gz to data


## General Functions

The following cells define functions that we will use in the rest of the code. You do not need to do anything here.

In [8]:
concat = lambda x: np.concatenate(x, axis=0)
to_np = lambda x: x.data.cpu().numpy() # Converting tensor on GPU (to firstly CPT) to a numpy array 

'''
Calculates the anomaly scores for a portion of the given dataset. 
If a GPU is not available, will run on a smaller fraction of the
dataset, so that calculations will be faster.

loader: A DataLoader that contains the loaded data of a dataset
anomaly_score_calculator: A function that takes in the output 
                          logit of a batch of data and/or the 
                          penultimate.
model_net: The classifier model.
usePenultimate: True if anomaly_score_calculator needs the 
                penultimate as a parameter. False otherwise.
'''
def get_ood_scores(loader, anomaly_score_calculator, model_net, use_penultimate = False):
    _score = []

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(loader):
            if torch.cuda.is_available():
                fraction = 200
            else:
                fraction = 1000
            if batch_idx >= ood_num_examples // fraction:
                break

            if torch.cuda.is_available():
                data = data.cuda()

            output = model_net(data)

            if use_penultimate:
                score = anomaly_score_calculator(output[0], output[1])
            else:
                score = anomaly_score_calculator(output[0])
            _score.append(score)
            # print(f"Appended score shape: {score.shape}")  # This was to ensure that 
    return concat(_score).copy()

In [9]:
# /////////////// Printing Results ///////////////
all_anomaly_results = {}

'''
Returns and prints out the AUROC score of a dataset.

ood_loader: A DataLoader that contains the loaded data of a dataset
anomaly_score_calculator: A function that takes in the output 
                          logit of a batch of data and/or the 
                          penultimate.
model_net: The classifier model.
usePenultimate: True if anomaly_score_calculator needs the 
                penultimate as a parameter. False otherwise.
'''
def get_and_print_results(ood_loader, anomaly_score_calculator, model_net, use_penultimate):
    out_score = get_ood_scores(ood_loader, anomaly_score_calculator, model_net, use_penultimate = use_penultimate)
    auroc = get_auroc(out_score, in_score)
    print('AUROC: \t\t\t{:.2f}'.format(100 * auroc) + "%")
    return auroc

'''
Prints out the AUROC score of all the OOD datasets. The results 
will be appended to global variable all_anomaly_results, which 
is used later for display purposes.

anomaly_score_calculator: A function that takes in the output 
                          logit of a batch of data and/or the 
                          penultimate.
anomaly_score_name: The name of the anomaly score method.
model_net: The classifier model.
model_name: The name of the classifier model.
usePenultimate: True if anomaly_score_calculator needs the 
                penultimate as a parameter. False otherwise.
'''
def print_all_results(anomaly_score_calculator, anomaly_score_name, model_net, model_name = "default_model", use_penultimate = False):
    global in_score, all_anomaly_results
    in_score = get_ood_scores(cifar10_loader, anomaly_score_calculator, model_net, use_penultimate)
    results = []

    print('Rademacher Noise Detection')
    auroc = get_and_print_results(rademacher_ood_loader, anomaly_score_calculator, model_net, use_penultimate)
    results.append(auroc)

    print('\nSVHN Detection')
    auroc = get_and_print_results(svhn_ood_loader, anomaly_score_calculator, model_net, use_penultimate)
    results.append(auroc)

    # print('\nDTD Detection')
    # auroc = get_and_print_results(dtd_ood_loader, anomaly_score_calculator, model_net, use_penultimate)
    # results.append(auroc)

    print('\nCIFAR-100 Detection')
    auroc = get_and_print_results(cifar100_ood_loader, anomaly_score_calculator, model_net, use_penultimate)
    results.append(auroc)

    average = sum(results) / len(results)
    results.append(average)

    if not model_name in all_anomaly_results:
        all_anomaly_results[model_name] = {}
    all_anomaly_results[model_name][anomaly_score_name] = results

## Implement AUROC Score
Fill in the get_auroc score. We will use this function in order to calculate the AUROC score of an out-of-distribution dataset.

It may be helpful to use the sklearn.metrics.roc_auc_score() function. Both _pos and _neg should be used.

In [10]:
'''
Calculates the AUROC score of a OOD dataset.

_pos: an array of anomoly scores of the OOD dataset
_neg: an array of anomoly scores of images in the CIFAR-10 dataset
return: The AUROC score the data in decimal form
'''
def get_auroc(_pos, _neg):
    ############################################################################
    # TODO:  Calculate the AUROC score.                                        #
    ############################################################################
    # An ROC curve (receiver operating characteristic curve) is a graph showing the performance of a classification model at all classification thresholds. This curve plots two parameters: True Positive Rate. False Positive Rate.
    # AUROC refers to the "area underneath ROC curve", and AUROC score is a decimal point that reflect tells us how efficient the model is. The higher the AUC, the better the model's performance at distinguishing between the positive and negative classes.
    # AUROC score of an out-of-distribution dataset reflects how efficient the model is on out-of-distribution dataset
    # sklearn.metrics.roc_auc_score takes in the true labels and predicted labels and then calculate an AUROC score 

    # auroc_score = sk.roc_auc_score(_pos, _neg)
    # I might simply be wrong. I have no idea why should I put _pos and _neg here. 

    true_labels = np.concatenate([np.ones(len(_neg)), np.zeros(len(_pos))]) # Where 1 means labels to in-distrobution labels while 0 means labels to OOD labels 
    probability_estimates = np.concatenate([_neg, _pos]) # It seems both true labels and predicted labels are from the same data sources? Very interresting & somehow makes sense to me 

    auroc_score = sk.roc_auc_score(true_labels, probability_estimates) # Note that an error here once incurred repetitive reference of this function by itself when you wrote auroc_score = get_auroc(true_labels, probability_estimates)
    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    return auroc_score # Should be a numpy array, given all above is. 

## Implement Anomaly Score Calculators
Fill in the folllowing functions which calculate the anomaly score given the model's output for a batch of data (the output will contain one logit per image in the batch).

The following equations show how the logits should be transformed in order to get the anomaly score.

Max Logit Equation: <br>
$\text{Score}=-\text{max} l_k$

Max Softmax Equation: <br>
$\text{Score}=-\text{max} p(y=k|x)$

Cross Entropy Anomaly Equation: <br>
$\text{Score} = \bar{l}-\text{log}∑_{c=1}^{\text{num_classes}}e^{l_c}$

In [11]:
# /////////////// Anomaly Score Calculators ///////////////

# 这里我的一些问题：
# 我不能理解anomaly detection score calculator的目的和流程
# 对于提供好的公式，我不清楚每一个notations都代指什么

'''
Calculates the max logit anomaly score of a batch of outputs.

output: The model's output for a batch of data.
'''
def max_logit_anomaly_score(output):
    ############################################################################
    # TODO: Calculate the max logit anomaly score (returning a numpy array).   #
    ############################################################################
    # Max logit equation 
    # $l_k$ refers to the logit (log-odds) values by model for each class. "odds"= P(events happening)/P(events not happenning). Log-odds simply means taking a natural log. 
    # print(f'dim of output is {output.shape}')
    score, index = torch.max(output, dim = 1) # Calculate the maximum out of a tensor along its second axis. The output will contain one logit per image in the batch. Score should be a tensor of (200,1) dimension
    # torch.max produces both max and its index 
  
    score = to_np (score) # Move the tensor to CPU before converting it to a numpy array

    score = - score # As instructed 
     
    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    return score

'''
Calculates the max softmax anomaly score of a batch of outputs.

output: The model's output for a batch of data.
'''
def max_softmax_anomaly_score(output):
    ############################################################################
    # TODO: Calculate the max softmax anomaly score (returning a numpy array). #
    ############################################################################
    # Max softmax equation
    # First you'll need to convert logits into probabilities. To do so, you will need to calculate softmax. 
    probabilities = F.softmax(output, dim=1) # dim=1 because you want to calculate the softmax along the dim of class and that was the sec dimension. 
    score, index = torch.max(probabilities, dim=1)  # Calculate the maximum along the dim of class 

    score = to_np (score)
    soure = - score 
    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    return score

'''
Calculates the cross entropy anomaly score of a batch of outputs.

output: The model's output for a batch of data.
'''
def cross_entropy_anomaly_score(output):
    ############################################################################
    # TODO: Calculate the cross entropy anomaly score (returning a numpy array).#
    ############################################################################
    # Calculate l_bar (average logit values)
    l_bar = torch.mean(output)  #l_bar is the average of all logit values

    # Calculate the log_sum 
    log_sum = torch.log(torch.sum(torch.exp(output), dim=1)) # Note that you want to sum up along the sec dim

    # Calculate cross_entropy_anomaly_score 
    score = l_bar - log_sum

    score = to_np(score)

    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    return score

## Print AUROC Results

Run the following cells in order to see how well each of the anomaly score calculators do on the OOD datasets.

In [12]:
print("======= Max Logit AUROC Scores =======")
print_all_results(max_logit_anomaly_score, "Max Logit", net)

======= Max Logit AUROC Scores =======
Rademacher Noise Detection
AUROC: 			29.13%

SVHN Detection
AUROC: 			9.16%

CIFAR-100 Detection
AUROC: 			12.58%


In [13]:
print("======= Max Softmax AUROC Scores =======")
print_all_results(max_softmax_anomaly_score, "Max Softmax", net)

======= Max Softmax AUROC Scores =======
Rademacher Noise Detection
AUROC: 			80.56%

SVHN Detection
AUROC: 			91.62%

CIFAR-100 Detection
AUROC: 			87.98%


In [14]:
print("======= Cross Entropy AUROC Scores =======")
print_all_results(cross_entropy_anomaly_score, "Cross Entropy", net)

======= Cross Entropy AUROC Scores =======
Rademacher Noise Detection
AUROC: 			29.09%

SVHN Detection
AUROC: 			8.67%

CIFAR-100 Detection
AUROC: 			12.61%


## Implement ViM

We will now implement virtual-logit matching (ViM). ViM works by doing the following. 

ViM first considers the **principal space** ($P$) of the features obtained by passing the training data (CIFAR-10) through to just before the final fully-connected layer. These features are then centered at the origin of a new coordinate system (defined by `u` in our code below). Let us use the 12 most significant principal components of these features for our principal space.

ViM calculates **alpha** ($\alpha$) by projecting the training data's features onto the space orthogonal to the prinicpal space ($P^\perp$). This projection is known as the **residual** ($x^{P^\perp}$). HINT: You may find it helpful to use np.linalg.svd to find the principal components (or you can use eignvalues and eigenvectors). The space orthogonal to the principle space can be found simply by taking the remaining principal components (i.e. all PCs apart from the first 12).

Alpha is then calculated by the following equation:
> $\alpha := \frac{∑^k_{i=1}\text{max}_{j=1,...,C}\{l^i_j\}}{∑^K_{i=1}\| x_i^{P\perp} \|}$

In other words, alpha is the sum of each logit's maximum value, divided by the sum of the norms of each feature's residuals.

Your calculated alpha in this part should be around 16.16.


In [15]:
# You may find the following functions useful.
from numpy.linalg import pinv, norm
from scipy.special import logsumexp

_score = []
to_np = lambda x: x.data.cpu().numpy()
concat = lambda x: np.concatenate(x, axis=0)

# Extraction fully connected layer's weights and biases
w, b = net.fc.weight.cpu().detach().numpy(), net.fc.bias.cpu().detach().numpy()
# Origin of a new coordinate system of feature space to remove bias


'''
Calculates and returns the space orthogonal to the principal space (principal_space_perp)
and alpha values given the training data the model used.


training_data_loader: A DataLoader that contains the loaded data of a 
                      training dataset.
model_net: The classifier model.
verbose: If true, will print out the alpha value.
return: Returns the alpha value.
'''
def compute_ViM_principal_space_perp_and_alpha(training_data_loader, model_net, verbose = False):
    result = []




    # Getting the first batch of the training data to calculate principal_space_perp and alpha
    # Principle space: the 12 most significant components of features 
    # Residuals: The projection/remaining/space orthogonal to the PS: all remaining components
    # Steps (i) PS (ii) Residuals (iii) alpha 
    training_data, target = next(iter(training_data_loader))
    if torch.cuda.is_available():
        training_data = training_data.cuda()
    # Step 1: Passing the training data (CIFAR-10) through to just before the final fully-connected layer
    result = model_net(training_data)

    logit = result[0] # Logits (values before softmax) # Interesting. This time the logits are not given as "output", as it was for Anomaly Scores
    penultimate = result[1] # Features/Penultimate (values before fully connected layer) # I think this penultimate is what is takes to calculate PS. 

    logit_id_train = logit.cpu().detach().numpy().squeeze()  # to CPU --> get rid of the gradients --> to numpy --> get rid of redundant dimensions
    feature_id_train = penultimate.cpu().detach().numpy().squeeze() 


    # PCA算法相关的步骤我好像还不太懂
    ############################################################################
    # TODO:  Calculate the space orthogonal to the pricipal space and then     #
    # compute alpha.                                                           #
    ############################################################################
    if verbose:  # What is the "verbose" condition?
        u = np.mean(feature_id_train, axis=0)  # Adjust the center for coordinate 
        # Step 2: Features are then centered at the origin of a new coordinate system (defined by u in our code below)
        centered_features = feature_id_train - u # My intuition is +u because u itself contains "-"
        # Step 3: Use the 12 most significant principal components of these features for our principal space.
        # Hint Use np.linalg.svd to find the principal components (or you can use eignvalues and eigenvectors).
        print('Computing principal_space_perp...')  # This is residuals. I think they unnecessarily introduced too many concepts that say the same things. 
        u, s, vh = np.linalg.svd(centered_features, full_matrices=True, compute_uv=True, hermitian=False)   # HINT: You may find it helpful to use np.linalg.svd to find the principal components (or you can use eignvalues and eigenvectors).
        # Taking the first 12 components 
        principal_space = vh[:12] # Let us use the 12 most significant principal components of these features for our principal space.

        # Project the centered features onto the principal space:
        projected_features = centered_features @ principal_space.T

        # Reconstruct the features from the projected features using the principal space:
        reconstructed_features = projected_features @ principal_space


        # Step 4: Residuals 
        principal_space_perp = centered_features - reconstructed_features # " The space orthogonal to the principle space can be found simply by taking the remaining principal components (i.e. all PCs apart from the first 12)."

    if verbose:
        print('Computing alpha...')
        # Step 5: Calculating alpha 
        alpha = np.sum(np.amax(logit_id_train, axis=1)) / np.sum(np.linalg.norm(principal_space_perp,axis=1)) #


    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    if verbose:
        print(f'alpha = {alpha}')

    return principal_space_perp, alpha

principal_space_perp, alpha = compute_ViM_principal_space_perp_and_alpha(cifar10_loader, net, verbose = True)

Computing principal_space_perp...
Computing alpha...
alpha = 16.28367042541504


## Implement ViM Anomaly Score Calculator

Now, implement the ViM anomaly score calculator. 

First, we want to project our penultimate/feature values onto principal_space_perp which we found before, which is called the residual. We multiply the norm of this residual by alpha to get what we call the **virtual logit score**. 

Next, we get what some call the **energy score** by taking LogSumExp of the logits. 

Finally, the outputted **anomaly score** is calculated by subtracting the virtual logit by the energy score.

$\text{vlogit}= \alpha \| {x^{P^\perp}} \|$\
$\text{energy} = \text{ln}\sum_{i=1}^C e^{l_i}$\
$\text{anomaly_score} = \text{vlogit} - \text{energy}$

In [19]:
'''
Calculates the ViM anomaly score of a batch of outputs.

output: The model's output for a batch of data.
penultimate: The model's penultimate (feature values) for a batch of data.
'''
def ViM_anomaly_score_calculator(output, penultimate):
    logit_id_val = output.cpu().detach().numpy().squeeze()
    feature_id_val = penultimate.cpu().detach().numpy().squeeze()

    ############################################################################
    # TODO:  Calculate the anomaly score.                                      #
    ############################################################################
    # virtual logir score  # It's kinda weird to me that they offered different variables for different functions. Sometimes I need to do something repetitively accordingly. Either side should be wrong. 
    residuals_norm = torch.norm(torch.tensor(principal_space_perp), dim=1)  # Cal the norms along the dim of feature space
    vlogit = alpha * residuals_norm
    # energy score 
    energy = np.log(np.sum(np.exp(logit_id_val)))   
    # anomaly score  # I didn't get it why we are calculating anomaly score once again. 
    anomaly_score = vlogit - energy 

    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    
    return anomaly_score

print("======= ViM_anomaly_score_calculator =======")
w, b = net.fc.weight.cpu().detach().numpy(), net.fc.bias.cpu().detach().numpy()
u = -np.matmul(pinv(w), b)
principal_space_perp, alpha = compute_ViM_principal_space_perp_and_alpha(cifar10_loader, net, verbose = True) # Making sure you have the correct ViM values before calculating the score 
print_all_results(ViM_anomaly_score_calculator, "ViM", net, use_penultimate = True)

======= ViM_anomaly_score_calculator =======
Computing principal_space_perp...
Computing alpha...
alpha = 16.28367042541504
Rademacher Noise Detection
AUROC: 			30.95%

SVHN Detection
AUROC: 			21.30%

CIFAR-100 Detection
AUROC: 			23.93%


## Compare Anomaly Score Results
Run the following cell to see how the different anomaly score calculators compare to each other for the OOD datasets. You should see that ViM is superior to other anomaly scores in all of the datasets except for CIFAR-10.

In [20]:
# ///////////////// Compare Results /////////////////

def get_results_max(model_name = "normal"):
    all_anomaly_results[model_name]["max"] = [0,0,0,0,0]
    for key in all_anomaly_results[model_name].keys():
        if (key != "max"):
            index = 0
            for score in all_anomaly_results[model_name][key]:
                all_anomaly_results[model_name]["max"][index] = \
                    max(score, all_anomaly_results[model_name]["max"][index])
                index += 1


def compare_all_results():
    for model_name in all_anomaly_results:
        to_be_printed = " " * (25 - len(model_name)) + model_name
        dataset_names = ["Rademacher", "SVHN", "CIFAR-100", "Average"]
        for name in dataset_names:
            to_be_printed += " | " + " "*(6-math.ceil(len(name)/2)) + \
                                name + " "*(6-math.floor(len(name)/2))

        print(to_be_printed)
        print("=" * (25 + len(dataset_names) * 15))

        get_results_max(model_name = model_name)
        for key in all_anomaly_results[model_name].keys():
            if (key != "max"):
                to_be_printed = " "*(25-len(key)) + key
                index = 0
                for result in all_anomaly_results[model_name][key]:
                    if (all_anomaly_results[model_name]["max"][index] == result):
                        result = "*" + '{:.2f}'.format(round(result * 100, 2)) + "%"
                    else:
                        result = '{:.2f}'.format(round(result * 100, 2)) + "%"
                    to_be_printed += " | " + " "*(6-math.ceil(len(result)/2)) + \
                                        result + " "*(6-math.floor(len(result)/2))
                    index += 1
                print(to_be_printed)
        print()

    print("\n* highlights the maximum AUROC Score for an OOD Dataset")

compare_all_results()

            default_model |  Rademacher  |     SVHN     |  CIFAR-100   |   Average   
                Max Logit |    29.13%    |    9.16%     |    12.58%    |    16.96%   
              Max Softmax |   *80.56%    |   *91.62%    |   *87.98%    |   *86.72%   
            Cross Entropy |    29.09%    |    8.67%     |    12.61%    |    16.79%   
                      ViM |    30.95%    |    21.30%    |    23.93%    |    25.39%   


* highlights the maximum AUROC Score for an OOD Dataset


## Data Augmentation

Now we will see if models trained using data augmentation methods for robustness help in OOD detection. 

We will load a CIFAR-10 model that used PixMix data augmentation during training, and see how it fares compared to the default model that did not use data augmentation.

In [21]:
!gdown 1mjIfbb3mfXXvAZ1sBnjotFr5yYFmLi68 # Downloading PixMix model
!gdown 1skZT6yplO-Sv4M8Ksgzx14cTY3H-HgOa # Downlaoding wideresnet class

Downloading...
From: https://drive.google.com/uc?id=1mjIfbb3mfXXvAZ1sBnjotFr5yYFmLi68
To: /content/checkpoint.pth.tar
100% 71.8M/71.8M [00:00<00:00, 73.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1skZT6yplO-Sv4M8Ksgzx14cTY3H-HgOa
To: /content/wideresnet_with_pen.py
100% 4.03k/4.03k [00:00<00:00, 25.0MB/s]


In [22]:
from wideresnet_with_pen import WideResNet as WideResNet2

In [23]:
# Loading the PixMix model

pixmix_net = WideResNet2(depth=40, num_classes=10, widen_factor=4, drop_rate=0.3)
pixmix_net = torch.nn.DataParallel(pixmix_net)

if torch.cuda.is_available():
    checkpoint = torch.load('checkpoint.pth.tar')
    pixmix_net.load_state_dict(checkpoint['state_dict'])
    net.eval()
    net.cuda()
else:
    checkpoint = torch.load('checkpoint.pth.tar', map_location=torch.device('cpu'))
    pixmix_net.load_state_dict(checkpoint['state_dict'])
    net.eval()

## PixMix Model Testing

Let us now test the PixMix model with the same anomaly score calculators we coded before.

In [24]:
print("======= Max Logit AUROC Scores =======")
print_all_results(max_logit_anomaly_score, "Max Logit", pixmix_net, model_name = "pixmix_trained_model")

======= Max Logit AUROC Scores =======
Rademacher Noise Detection
AUROC: 			5.59%

SVHN Detection
AUROC: 			7.36%

CIFAR-100 Detection
AUROC: 			12.52%


In [25]:
print("======= Max Softmax Probability AUROC Scores =======")
print_all_results(max_softmax_anomaly_score, "Max Softmax Probability", pixmix_net, model_name = "pixmix_trained_model")

======= Max Softmax Probability AUROC Scores =======
Rademacher Noise Detection
AUROC: 			92.81%

SVHN Detection
AUROC: 			91.36%

CIFAR-100 Detection
AUROC: 			87.57%


In [26]:
print("======= Cross Entropy AUROC Scores =======")
print_all_results(cross_entropy_anomaly_score, "Cross Entropy", pixmix_net, model_name = "pixmix_trained_model")

======= Cross Entropy AUROC Scores =======
Rademacher Noise Detection
AUROC: 			5.77%

SVHN Detection
AUROC: 			7.88%

CIFAR-100 Detection
AUROC: 			13.62%


In [28]:
print("======= ViM_anomaly_score_calculator =======")
w, b = pixmix_net.module.fc.weight.cpu().detach().numpy(), pixmix_net.module.fc.bias.cpu().detach().numpy()
u = -np.matmul(pinv(w), b)
principal_space_perp, alpha = compute_ViM_principal_space_perp_and_alpha(cifar10_loader, pixmix_net, verbose = True)
print_all_results(ViM_anomaly_score_calculator, "ViM", pixmix_net, model_name = "pixmix_trained_model", use_penultimate = True)

======= ViM_anomaly_score_calculator =======
Computing principal_space_perp...
Computing alpha...
alpha = 12.416947364807129
Rademacher Noise Detection
AUROC: 			5.31%

SVHN Detection
AUROC: 			8.22%

CIFAR-100 Detection
AUROC: 			18.00%


## Compare No-data-augmentation Model vs PixMix Model

Let us now compare how the default model compared to the PixMix model by running the following cell. 

You should see that the PixMix model successfully helps us in OOD detection, and has a higher AUROC score (except for ViM). This suggests that one should evaluate models across numerous OOD datasets to soundly assess OOD detection performance.

In [29]:
compare_all_results()

            default_model |  Rademacher  |     SVHN     |  CIFAR-100   |   Average   
                Max Logit |    29.13%    |    9.16%     |    12.58%    |    16.96%   
              Max Softmax |   *80.56%    |   *91.62%    |   *87.98%    |   *86.72%   
            Cross Entropy |    29.09%    |    8.67%     |    12.61%    |    16.79%   
                      ViM |    30.95%    |    21.30%    |    23.93%    |    25.39%   

     pixmix_trained_model |  Rademacher  |     SVHN     |  CIFAR-100   |   Average   
                Max Logit |    5.59%     |    7.36%     |    12.52%    |    8.49%    
  Max Softmax Probability |   *92.82%    |   *91.36%    |   *87.57%    |   *90.58%   
            Cross Entropy |    5.77%     |    7.88%     |    13.62%    |    9.09%    
                      ViM |    5.31%     |    8.22%     |    18.00%    |    10.51%   


* highlights the maximum AUROC Score for an OOD Dataset


In [30]:
# Your output above should look similar to below